# Google Colab for training

### Set up the files inside colab

Make sure you have a `ML-For-Context-In-AI-Assistant-colab.zip` in your google disk before funning the cell bellow

In [1]:
# imports
import zipfile
import shutil
import os

from google.colab import drive

# 1. Mount Google Drive
drive.mount('/content/drive')

# Path to the zip file in your mounted Drive
zip_path = '/content/drive/MyDrive/ML-For-Context-In-AI-Assistant-colab.zip'

# Destination directory
extract_to = '/content/'

# Make sure the zip exists
if not os.path.isfile(zip_path):
    raise FileNotFoundError(f"Could not find zip file at {zip_path}")

# Unzip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

# source and destination
src_dir = '/content/ML-For-Context-In-AI-Assistant-colab'
dst_dir = '/content'

# sanity check
if not os.path.isdir(src_dir):
    raise FileNotFoundError(f"{src_dir} does not exist")

# move each file/subfolder
for name in os.listdir(src_dir):
    src_path = os.path.join(src_dir, name)
    dst_path = os.path.join(dst_dir, name)

    if os.path.exists(dst_path):
        print(f"Warning: {dst_path} already exists, overwriting")
        if os.path.isdir(dst_path):
            shutil.rmtree(dst_path)
        else:
            os.remove(dst_path)
    shutil.move(src_path, dst_path)

# remove the empty folder
os.rmdir(src_dir)

### Helper Funcs:

def zip_folder(folder_path: str, output_path: str) -> None:
    """
    Recursively zip the contents of folder_path into a .zip file at output_path.

    :param folder_path: Path to the folder to compress.
    :param output_path: Path (including filename) for the output .zip file.
    """
    with zipfile.ZipFile(output_path, 'w', compression=zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                # Preserve folder structure in the archive
                arcname = os.path.relpath(file_path, start=folder_path)
                zipf.write(file_path, arcname)

Mounted at /content/drive


### Install the requirements

If the message of restarting the kernel appears ignore it as i did (otherwise, do your own magic) :]

In [2]:
# --- Prepare: upgrade pip tooling & install uv ---
!python -m pip install -q -U pip wheel setuptools uv

# --- Install PyTorch first (GPU if available, else CPU) ---
import subprocess, os
has_gpu = "NVIDIA" in subprocess.getoutput("nvidia-smi -L 2>/dev/null")
index = "https://download.pytorch.org/whl/cu121" if has_gpu else "https://download.pytorch.org/whl/cpu"
print("GPU:", has_gpu, "| Torch index:", index)

# Pin torch first; change version if you really need 2.8.0
!pip install -q -U --index-url {index} torch==2.8.0

# --- Use uv to install everything else from your file (faster resolver/downloads) ---
# If your file is named differently, adjust the path:
req_path = "/content/requirements.txt"  # put your pasted contents here or upload_file
filtered = []
for line in open(req_path):
    if line.strip().startswith("torch=="):  # we already installed torch
        continue
    if line.strip().startswith("ipython=="):  # we already installed torch
        continue
    if line.strip().startswith("ipykernel=="):  # we already installed torch
        continue
    if line.strip().startswith("tornado=="):  # we already installed torch
        continue
    if line.strip().startswith("prompt_toolkit=="):  # we already installed torch
        continue
    if line.strip().startswith("pyzmq=="):  # we already installed torch
        continue

    filtered.append(line)
open("/content/requirements_no_torch.txt","w").write("".join(filtered))

# Install into the current environment (system Python inside Colab)
!pip install -r requirements_no_torch.txt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 90.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.
GPU: False | Torch index: https://download.pytorch.org/whl/cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 99.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 57.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 97.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 193.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 46.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 156.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━

### Seting up the Qdrant DB (parallel and backend)

In [3]:
# Clean any old processes/logs
!pkill -f qdrant || true
!rm -f /content/qdrant.log

# Download & unpack a Qdrant Linux binary
!wget -qO /content/qdrant.tar.gz https://github.com/qdrant/qdrant/releases/download/v1.10.0/qdrant-x86_64-unknown-linux-gnu.tar.gz
!tar -xzf /content/qdrant.tar.gz -C /content
!chmod +x /content/qdrant

# Make a storage dir (optional; Qdrant will default to ./storage anyway)
!mkdir -p /content/storage

# Start Qdrant (NO --storage flag). It will bind to 0.0.0.0:6333 and use ./storage.
!nohup /content/qdrant --uri http://0.0.0.0:6333 > /content/qdrant.log 2>&1 &

# Give it a second to boot
import time; time.sleep(2)

# Sanity checks
!ps -ef | grep qdrant | grep -v grep || true
!curl -s http://127.0.0.1:6333/ | head -n 5 || true

# If curl didn't return JSON, print logs:
!sed -n '1,200p' /content/qdrant.log

^C
root        1046       1  1 17:51 ?        00:00:00 /content/qdrant --uri http://0.0.0.0:6333
{"title":"qdrant - vector search engine","version":"1.10.0","commit":"851f03bbf6644116da56f6bc7b0baa04274e8057"}           _                 _    
  __ _  __| |_ __ __ _ _ __ | |_  
 / _` |/ _` | '__/ _` | '_ \| __| 
| (_| | (_| | | | (_| | | | | |_  
 \__, |\__,_|_|  \__,_|_| |_|\__| 
    |_|                           

Version: 1.10.0, build: 851f03bb
Access web UI at http://localhost:6333/dashboard

2025-10-16T17:51:34.242154Z  WARN qdrant::settings: Config file not found: config/config    
2025-10-16T17:51:34.242165Z  WARN qdrant::settings: Config file not found: config/development    
2025-10-16T17:51:34.242204Z  INFO storage::content_manager::consensus::persistent: Initializing new raft state at ./storage/raft_state.json    
2025-10-16T17:51:34.249524Z  INFO qdrant: Distributed mode disabled    
2025-10-16T17:51:34.249546Z  INFO qdrant: Telemetry reporting enabled, id: 3d4b9f34-fe5e-4

### Evaluate the baseline model

In [6]:
%env PYTHONPATH=/content:$PYTHONPATH

!python scripts/evaluate.py \
    --model "sentence-transformers/all-MiniLM-L12-v2" \
    --qdrant-host "127.0.0.1" \
    --qdrant-port 6333 \
    --qdrant-collection "cosqa_test_bodies" \
    --K 3

env: PYTHONPATH=/content:$PYTHONPATH
2025-10-16 17:52:20.917519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760637140.936926    1237 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760637140.942773    1237 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760637140.957595    1237 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760637140.957619    1237 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760637140.957622    1237 computation_

### Fine-tuning of the baseline model

In [9]:
!python scripts/train.py \
    --model "sentence-transformers/all-MiniLM-L12-v2" \
    --finetune-dir "./models" \
    --checkpoint-path "./checkpoint" \
    --assets-dir "./results/assets" \
    --qdrant-host "qdrant" \
    --qdrant-port 6333 \
    --qdrant-collection "cosqa_test_bodies" \
    --qdrant-collection-ft "cosqa_test_ft" \
    --K 10 \
    --batch-size 32 \
    --epochs 10 \
    --lr 2e-5 \
    --max-steps-per-epoch 0 \
    --seed 69

2025-10-16 17:55:07.743954: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760637307.763991    2141 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760637307.769833    2141 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760637307.784798    2141 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760637307.784822    2141 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760637307.784826    2141 computation_placer.cc:177] computation placer alr

### Final evaluation of fine-tuned model

In [12]:
!python scripts/evaluate.py \
    --model "../models" \
    --qdrant-host "127.0.0.1" \
    --qdrant-port 6333 \
    --qdrant-collection "cosqa_test_bodies" \
    --K 3

2025-10-16 18:14:03.411031: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760638443.442029    7233 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760638443.451495    7233 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1760638443.473963    7233 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760638443.473990    7233 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1760638443.473998    7233 computation_placer.cc:177] computation placer alr

### Saving of model

In [13]:
zip_folder("../models", "./models.zip")